# **DMB-S9 - TP3 / Mini-Projet Spark : Catastrophes Naturelles aux Etats-Unis 🌪 🚑**
### *Yazid BENJAMAA & Thomas DELAPART - ESIR  SI*

In [54]:
!uname -a
!pip install graphframes
!pip install pyspark

Linux ae2185cd2944 6.1.58+ #1 SMP PREEMPT_DYNAMIC Sat Nov 18 15:31:17 UTC 2023 x86_64 x86_64 x86_64 GNU/Linux


In [91]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import datediff, avg, min, desc, year, when
import plotly.express as px
import plotly.graph_objs as go

In [56]:
spark = SparkSession.builder \
    .appName("Temporal_Trends_Analysis") \
    .getOrCreate()

<h1><b>I. Le jeu de données </b></h1>

Le dataset que nous avons choisi répertorie l'historique des catastrophes naturelles aux Etats-Unis de 1953 à 2023.

Source (Kaggle): <a href="https://www.kaggle.com/datasets/headsortails/us-natural-disaster-declarations?select=us_disaster_declarations.csv"><b>US Natural Disaster Declarations</b></a>

Description : This summary dataset is a high-level summary of all federally declared disasters since 1953, `us_disaster_declarations.csv` provides the geographical resolution is the county level, with FIPS codes being used to encode the counties. In addition to the fips and timing features, the data provides the type of disaster and also binary flags that indicate whether specific aid programs were triggered in response.


<table border="1">
  <tr>
    <th>Nom du champ</th>
    <th>Description</th>
  </tr>
  <tr>
    <td>fema_declaration_string</td>
    <td>Méthode standard de l'agence pour identifier de manière unique les déclarations de la loi Stafford. Concaténation du type de déclaration, du numéro de catastrophe et de l'état.</td>
  </tr>
  <tr>
    <td>disaster_number</td>
    <td>Numéro attribué séquentiellement pour désigner un événement ou un incident déclaré comme catastrophe.</td>
  </tr>
  <tr>
    <td>state</td>
    <td>État, district ou territoire des États-Unis.</td>
  </tr>
  <tr>
    <td>declaration_type</td>
    <td>L'un des "DR" (= grande catastrophe), "EM" (= gestion des urgences) ou "FM" (= gestion des incendies).</td>
  </tr>
  <tr>
    <td>declaration_date</td>
    <td>Date à laquelle la catastrophe a été déclarée.</td>
  </tr>
  <tr>
    <td>fy_declared</td>
    <td>Année fiscale au cours de laquelle la catastrophe a été déclarée.</td>
  </tr>
  <tr>
    <td>incident_type</td>
    <td>Type d'incident tel que "Incendie", "Inondation" ou "Ouragan". Le type d'incident affectera les types d'assistance disponibles.</td>
  </tr>
  <tr>
    <td>declaration_title</td>
    <td>Titre de la catastrophe. Cela peut être un identifiant utile tel que "Ouragan Katrina" ou "Pandémie de Covid-19".</td>
  </tr>
  <tr>
    <td>ih_program_declared</td>
    <td>Indicateur binaire indiquant si le programme "Individus et Foyers" a été déclaré pour cette catastrophe.</td>
  </tr>
  <tr>
    <td>ia_program_declared</td>
    <td>Indicateur binaire indiquant si le programme "Assistance Individuelle" a été déclaré pour cette catastrophe.</td>
  </tr>
  <tr>
    <td>pa_program_declared</td>
    <td>Indicateur binaire indiquant si le programme "Assistance Publique" a été déclaré pour cette catastrophe.</td>
  </tr>
  <tr>
    <td>hm_program_declared</td>
    <td>Indicateur binaire indiquant si le programme "Atténuation des Risques" a été déclaré pour cette catastrophe.</td>
  </tr>
  <tr>
    <td>incident_begin_date</td>
    <td>Date à laquelle l'incident lui-même a commencé.</td>
  </tr>
  <tr>
    <td>incident_end_date</td>
    <td>Date à laquelle l'incident lui-même s'est terminé. Cette caractéristique présente environ 14% d'entrées NA.</td>
  </tr>
  <tr>
    <td>disaster_closeout_date</td>
    <td>Date à laquelle toutes les transactions financières pour tous les programmes sont terminées. Cette colonne présente 98% d'entrées NA.</td>
  </tr>
  <tr>
    <td>fips</td>
    <td>Code FIPS de comté à 5 chiffres ; utilisé pour identifier les comtés et les équivalents de comté aux États-Unis, dans le district de Columbia, dans les territoires américains, dans les zones périphériques des États-Unis et dans les États associés librement. Concaténé à partir des 2 colonnes source "fipsStateCode" et "fipsCountyCode".</td>
  </tr>
  <tr>
    <td>place_code</td>
    <td>Un système de code unique que FEMA utilise en interne pour reconnaître les emplacements qui prend les numéros '99' + le code FIPS de comté à 3 chiffres. Il y a des emplacements déclarés qui n'ont pas de codes FIPS de comté reconnus, auquel cas un identifiant unique a été attribué.</td>
  </tr>
  <tr>
    <td>designated_area</td>
    <td>Le nom ou la phrase décrivant le comté américain inclus dans la déclaration. Peut prendre la valeur "À l'échelle de l'État".</td>
  </tr>
  <tr>
    <td>declaration_request_number</td>
    <td>Identifiant unique attribué à la demande de déclaration de catastrophe.</td>
  </tr>
  <tr>
    <td>hash</td>
    <td>Hash MD5 des champs et des valeurs de l'enregistrement.</td>
  </tr>
  <tr>
    <td>last_refresh</td>
    <td>Date à laquelle l'enregistrement a été mis à jour pour la dernière fois par FEMA.</td>
  </tr>
  <tr>
    <td>id</td>
    <td>Identifiant unique attribué à l'enregistrement. Ces 4 dernières colonnes sont principalement utilisées pour la comptabilité.</td>
  </tr>
</table>

Echantillon des données (2 premières lignes) :

<table border="1">
  <tr>
    <th>fema_declaration_string</th>
    <th>disaster_number</th>
    <th>state</th>
    <th>declaration_type</th>
    <th>declaration_date</th>
    <th>fy_declared</th>
    <th>incident_type</th>
    <th>declaration_title</th>
    <th>ih_program_declared</th>
    <th>ia_program_declared</th>
    <th>pa_program_declared</th>
    <th>hm_program_declared</th>
    <th>incident_begin_date</th>
    <th>incident_end_date</th>
    <th>disaster_closeout_date</th>
    <th>fips</th>
    <th>place_code</th>
    <th>designated_area</th>
    <th>declaration_request_number</th>
    <th>last_ia_filing_date</th>
    <th>last_refresh</th>
    <th>hash</th>
    <th>id</th>
  </tr>
  <tr>
    <td>DR-1-GA</td>
    <td>1</td>
    <td>GA</td>
    <td>DR</td>
    <td>1953-05-02T00:00:00Z</td>
    <td>1953</td>
    <td>Tornado</td>
    <td>Tornado</td>
    <td>0</td>
    <td>1</td>
    <td>1</td>
    <td>1</td>
    <td>1953-05-02T00:00:00Z</td>
    <td>1953-05-02T00:00:00Z</td>
    <td>1954-06-01T00:00:00Z</td>
    <td>13000</td>
    <td>0</td>
    <td>Statewide</td>
    <td>53013</td>
    <td>NA</td>
    <td>2023-03-18T13:22:03Z</td>
    <td>48af1afcc4535aa910ddb5b85eebe047dc703a6b</td>
    <td>da5c8f17-c28f-4c41-8e06-fa8efc85aa4a</td>
  </tr>
  <tr>
    <td>DR-2-TX</td>
    <td>2</td>
    <td>TX</td>
    <td>DR</td>
    <td>1953-05-15T00:00:00Z</td>
    <td>1953</td>
    <td>Tornado</td>
    <td>Tornado & Heavy Rainfall</td>
    <td>0</td>
    <td>1</td>
    <td>1</td>
    <td>1</td>
    <td>1953-05-15T00:00:00Z</td>
    <td>1953-05-15T00:00:00Z</td>
    <td>1958-01-01T00:00:00Z</td>
    <td>48000</td>
    <td>0</td>
    <td>Statewide</td>
    <td>53003</td>
    <td>NA</td>
    <td>2023-03-18T13:22:03Z</td>
    <td>6bbba411d294ec521683e36313c8da2992e8a857</td>
    <td>b0b41f55-0d77-43ac-898c-3cc4a18231e6</td>
  </tr>
  <!-- Ajoutez les autres lignes de données ici -->
</table>

In [57]:
file_path = "/content/drive/MyDrive/TP_DMB/us_disaster_declarations.csv"

data = spark.read.csv(file_path, header=True, inferSchema=True)

<h1><b>II. Questions traitées :</b></h1>
<h3> 1. Tendances temporelles </h3>
<ul>
  <li>Comment évolue le nombre de catastrophes naturelles au fil des années ? </li>
  <li> Y a-t-il des années avec un nombre exceptionnellement élevé ou faible de déclarations de catastrophes ?</li>
</ul>
<h3> 2. Types de catastrophes </h3>
<ul>
  <li>Quels sont les types de catastrophes naturelles les plus fréquents dans les déclarations ?</li>
  <li> Y a-t-il des régions plus susceptibles à certains types de catastrophes que d'autres ?</li>
</ul>
<h3> 3. Répartition géographique </h3>
<ul>
  <li>Quelles régions ont le plus souvent des déclarations de catastrophes naturelles ?
</li>
</ul>
<h3>4. Durée et impact des catastrophes
</h3>
<ul>
  <li>Quelle est la durée moyenne d'une catastrophe, de son début à sa clôture ?
</li>
<li>Quels sont les types de déclarations qui ont le plus grand impact en termes de pertes économiques ?
</li>
</ul>
<h3>5. Réponses et secours</h3>
<ul>
  <li>Quelle est la durée moyenne entre la déclaration d'une catastrophe naturelle et l'octroi d'une aide fédérale ? </li>
  <li>Quels états sont les plus rapides en matière d'octroi de l'aide ?
</li>
</ul>
<h3>6. Exposition aux catastrophes naturelles</h3>
<ul>
  <li>Quelle catastrophe naturelle est la plus fréquente dans chaque état ?
</li>
  <li>Quelles ont été les années les plus marquées en catastrophes naturelles pour chaque état ?
</li>
  <li>Quels programmes d'assitance ont été activés dans chaque état ?</li>
</ul>



### 1. Tendances temporelles :

Comment évolue le nombre de déclarations de catastrophes naturelles au fil des années ?
Y a-t-il des années avec un nombre exceptionnellement élevé ou faible de déclarations de catastrophes ?

In [58]:
# On extrait l'année
data = data.withColumn("year", year(data["declaration_date"]))

declaration_count = data.groupBy("year").count().orderBy("year")

declaration_count.show()

max_declarations = declaration_count.orderBy("count", ascending=False).first()
min_declarations = declaration_count.orderBy("count").first()

print("Année avec le plus grand nombre de déclarations de catastrophes naturelles:", max_declarations["year"])
print("Nombre de déclarations:", max_declarations["count"])

print("\nAnnée avec le plus faible nombre de déclarations de catastrophes naturelles:", min_declarations["year"])
print("Nombre de déclarations:", min_declarations["count"])

+----+-----+
|year|count|
+----+-----+
|1953|   13|
|1954|   17|
|1955|   18|
|1956|   16|
|1957|   16|
|1958|    7|
|1959|    7|
|1960|   12|
|1961|   12|
|1962|   22|
|1963|   20|
|1964|  136|
|1965|  429|
|1966|  120|
|1967|  196|
|1968|  161|
|1969|  505|
|1970|  223|
|1971|  174|
|1972|  526|
+----+-----+
only showing top 20 rows

Année avec le plus grand nombre de déclarations de catastrophes naturelles: 2020
Nombre de déclarations: 9712

Année avec le plus faible nombre de déclarations de catastrophes naturelles: 1959
Nombre de déclarations: 7


### 2. Types de catastrophes :

Quels sont les types de catastrophes naturelles les plus fréquents dans les déclarations ?
Y a-t-il des régions plus susceptibles à certains types de catastrophes que d'autres ?

In [59]:
type_counts = data.groupBy("incident_type").count().orderBy(desc("count"))

print("Types de catastrophes naturelles les plus fréquents :")
type_counts.show(truncate=False)

region_type_counts = data.groupBy("state", "incident_type").count().orderBy("state", desc("count"))

print("Régions les plus susceptibles à certains types de catastrophes :")
region_type_counts.show(truncate=False)

Types de catastrophes naturelles les plus fréquents :
+-----------------+-----+
|incident_type    |count|
+-----------------+-----+
|Severe Storm     |17490|
|Hurricane        |13045|
|Flood            |10693|
|Biological       |7857 |
|Snowstorm        |3707 |
|Fire             |3615 |
|Severe Ice Storm |2942 |
|Tornado          |1569 |
|Drought          |1292 |
|Coastal Storm    |637  |
|Other            |307  |
|Freezing         |301  |
|Earthquake       |228  |
|Typhoon          |130  |
|Tropical Storm   |54   |
|Volcanic Eruption|51   |
|Winter Storm     |43   |
|Fishing Losses   |42   |
|Mud/Landslide    |37   |
|Dam/Levee Break  |13   |
+-----------------+-----+
only showing top 20 rows

Régions les plus susceptibles à certains types de catastrophes :
+-----+-------------+-----+
|state|incident_type|count|
+-----+-------------+-----+
|AK   |Biological   |121  |
|AK   |Severe Storm |74   |
|AK   |Flood        |50   |
|AK   |Fire         |30   |
|AK   |Freezing     |14   |
|AK   |

## 3. Répartition géographique :

Quelles régions ont le plus souvent des déclarations de catastrophes naturelles ?

In [60]:
regions_counts = data.groupBy("state").count().orderBy(desc("count"))
print("Régions avec le plus grand nombre de déclarations de catastrophes naturelles :")
regions_counts.show(truncate=False)

Régions avec le plus grand nombre de déclarations de catastrophes naturelles :
+-----+-----+
|state|count|
+-----+-----+
|TX   |5173 |
|MO   |2704 |
|KY   |2583 |
|VA   |2524 |
|LA   |2493 |
|OK   |2474 |
|FL   |2352 |
|GA   |2278 |
|NC   |2096 |
|PR   |1987 |
|MS   |1906 |
|IA   |1848 |
|KS   |1759 |
|AL   |1665 |
|TN   |1605 |
|CA   |1599 |
|AR   |1593 |
|MN   |1563 |
|NY   |1503 |
|NE   |1485 |
+-----+-----+
only showing top 20 rows



In [88]:
state_counts_pandas = regions_counts.toPandas()

fig = go.Figure(data=go.Choropleth(
    locations=state_counts_pandas['state'],
    z=state_counts_pandas['count'],
    locationmode='USA-states',
    colorscale='Reds',
    colorbar_title='Nombre de déclarations de catastrophes'
))

fig.update_layout(
    title_text='Déclarations de catastrophes naturelles par état aux États-Unis',
    geo_scope='usa',
    width=1500,
    height=800
)

fig.show()


## 4. Durée et impact des catastrophes :

Quelle est la durée moyenne d'une catastrophe, de son début à sa clôture ?
Quels sont les types de déclarations qui ont le plus grand impact en termes de pertes économiques ?


In [62]:
# Durée moyenne d'une catastrophe, de son début à sa clôture
data = data.withColumn("incident_begin_date", data["incident_begin_date"].cast("date"))
data = data.withColumn("disaster_closeout_date", data["disaster_closeout_date"].cast("date"))
data = data.withColumn("duration", datediff(data["disaster_closeout_date"], data["incident_begin_date"]))
avg_duration = data.select(avg("duration")).collect()[0][0]
print("Durée moyenne d'une catastrophe (en jours) :", avg_duration)

# Types de déclarations avec le plus grand impact en termes de pertes économiques
economic_losses = data.groupBy("incident_type").sum("pa_program_declared").orderBy(desc("sum(pa_program_declared)"))
print("Types de déclarations avec le plus grand impact en termes de pertes économiques :")
economic_losses.show(truncate=False)

Durée moyenne d'une catastrophe (en jours) : 2361.6756215243704
Types de déclarations avec le plus grand impact en termes de pertes économiques :
+-----------------+------------------------+
|incident_type    |sum(pa_program_declared)|
+-----------------+------------------------+
|Severe Storm     |15289                   |
|Hurricane        |12755                   |
|Flood            |9963                    |
|Biological       |7673                    |
|Snowstorm        |3682                    |
|Fire             |3435                    |
|Severe Ice Storm |2885                    |
|Drought          |1287                    |
|Tornado          |1257                    |
|Coastal Storm    |614                     |
|Other            |306                     |
|Earthquake       |184                     |
|Typhoon          |130                     |
|Tropical Storm   |54                      |
|Volcanic Eruption|51                      |
|Winter Storm     |43                      |


## 5. Réponses et secours :

Quelle est la durée moyenne entre la déclaration d'une catastrophe naturelle et l'octroi d'une aide fédérale ?
Quels états sont les plus rapides en matière d'octroi de l'aide ?

In [63]:
# Durée moyenne entre la déclaration d'une catastrophe naturelle et l'octroi d'une aide fédérale
data = data.withColumn("declaration_date", data["declaration_date"].cast("date"))
data = data.withColumn("last_ia_filing_date", data["last_ia_filing_date"].cast("date"))
data = data.withColumn("response_time", datediff(data["last_ia_filing_date"], data["declaration_date"]))
avg_response_time = data.select(avg("response_time")).collect()[0][0]
print("Durée moyenne entre la déclaration d'une catastrophe naturelle et l'octroi d'une aide fédérale (en jours) :", avg_response_time)

# Tendances dans la rapidité de la réponse en fonction du type de catastrophe
response_time_trends = data.groupBy("incident_type").agg(min("response_time")).orderBy(desc("min(response_time)"))
print("Tendances dans la rapidité de la réponse en fonction du type de catastrophe :")
response_time_trends.show(truncate=False)

Durée moyenne entre la déclaration d'une catastrophe naturelle et l'octroi d'une aide fédérale (en jours) : 339.131921365403
Tendances dans la rapidité de la réponse en fonction du type de catastrophe :
+-----------------+------------------+
|incident_type    |min(response_time)|
+-----------------+------------------+
|Biological       |1140              |
|Freezing         |181               |
|Volcanic Eruption|124               |
|Dam/Levee Break  |83                |
|Typhoon          |61                |
|Earthquake       |61                |
|Mud/Landslide    |61                |
|Severe Ice Storm |60                |
|Other            |60                |
|Flood            |60                |
|Fire             |60                |
|Terrorist        |60                |
|Coastal Storm    |60                |
|Hurricane        |59                |
|Tornado          |59                |
|Severe Storm     |-305              |
|Human Cause      |NULL              |
|Snowstorm       

In [86]:
processed_df = data.filter(col("last_ia_filing_date").isNotNull())

processed_df = processed_df.withColumn("last_ia_filing_timestamp", unix_timestamp("last_ia_filing_date"))

avg_processing_time = processed_df.groupBy("state") \
    .agg((avg(col("last_ia_filing_timestamp")) / 86400 / count("*")).alias("avg_processing_time_days"))

avg_processing_time.show()

avg_processing_time_pd = avg_processing_time.toPandas()

fig = px.choropleth(avg_processing_time_pd,
                    locations='state',
                    locationmode="USA-states",
                    color='avg_processing_time_days',
                    scope="usa",
                    labels={'avg_processing_time_days':'Durée moyenne de traitement des demandes (jours)'},
                    title='Durée moyenne de traitement des demandes d\'assistance par État aux États-Unis')

fig.show()

+-----+------------------------+
|state|avg_processing_time_days|
+-----+------------------------+
|   SC|       70.64099017874337|
|   AZ|       330.5258875739645|
|   LA|      18.482951060877568|
|   MN|      54.181607142857146|
|   NJ|       106.9448377990652|
|   DC|       4832.666666666667|
|   OR|      222.03730584990564|
|   VA|       35.57675208289575|
|   RI|                 1067.75|
|   KY|      17.792476815874906|
|   WY|       651.6492271105826|
|   NH|                 382.075|
|   MI|       128.9839552818177|
|   NV|       417.3889413988658|
|   WI|      55.280946610204104|
|   ID|      269.01704142011835|
|   CA|       71.36115034248994|
|   NE|      54.229836333371246|
|   CT|        279.641498216409|
|   MT|       87.35916666666667|
+-----+------------------------+
only showing top 20 rows



## 6. Exposition aux catastrophes naturelles :
Quelle catastrophe naturelle est la plus fréquente dans chaque état ?

In [66]:
from pyspark.sql.functions import col, desc, row_number
from pyspark.sql.window import Window

incident_counts = data.groupBy("state", "incident_type") \
    .count() \
    .withColumnRenamed("count", "incident_count")

windowSpec = Window.partitionBy("state").orderBy(desc("incident_count"))

most_frequent_incidents = incident_counts.withColumn("rn", row_number().over(windowSpec)) \
    .filter(col("rn") == 1) \
    .drop("rn")

most_frequent_incidents.show()

+-----+-------------+--------------+
|state|incident_type|incident_count|
+-----+-------------+--------------+
|   AK|   Biological|           121|
|   AL|    Hurricane|           611|
|   AR| Severe Storm|           633|
|   AS|    Hurricane|            25|
|   AZ|         Fire|            88|
|   CA|         Fire|           460|
|   CO|         Fire|           151|
|   CT|    Hurricane|            96|
|   DC| Severe Storm|             9|
|   DE|    Hurricane|            22|
|   FL|    Hurricane|          1219|
|   FM|      Typhoon|            23|
|   GA|    Hurricane|           905|
|   GU|      Typhoon|            13|
|   HI|    Hurricane|            22|
|   IA|        Flood|           722|
|   ID|   Biological|            93|
|   IL|        Flood|           317|
|   IN| Severe Storm|           655|
|   KS| Severe Storm|          1030|
+-----+-------------+--------------+
only showing top 20 rows



In [74]:
most_frequent_incidents_pd = most_frequent_incidents.toPandas()

fig = px.choropleth(most_frequent_incidents_pd,
                    locations='state',
                    locationmode="USA-states",
                    color='incident_type',
                    scope="usa",
                    labels={'incident_type':'Type d\'incident le plus fréquent'})

fig.update_layout(
    title_text='Catastrophe naturelle la plus fréquente par État aux États-Unis',
    geo_scope='usa',
    width=1500,
    height=800
)


fig.show()

Quelles ont été les années les plus marquées en catastrophes naturelles pour chaque état ?

In [76]:
disaster_df = data.withColumn("year", year(col("declaration_date")))

disaster_counts = disaster_df.groupBy("state", "year").count()

windowSpec = Window.partitionBy("state").orderBy(desc("count"))

most_impactful_years = disaster_counts.withColumn("rn", row_number().over(windowSpec)) \
    .filter(col("rn") == 1) \
    .drop("rn")

most_impactful_years.show()

+-----+----+-----+
|state|year|count|
+-----+----+-----+
|   AK|2020|  121|
|   AL|2020|  278|
|   AR|2020|  240|
|   AS|2018|   10|
|   AZ|2020|   81|
|   CA|2020|  181|
|   CO|2020|  139|
|   CT|2011|   39|
|   DC|2011|    3|
|   DE|2020|    8|
|   FL|2022|  264|
|   FM|1992|    5|
|   GA|2017|  351|
|   GU|2002|    2|
|   HI|2018|   14|
|   IA|2020|  227|
|   ID|2020|   93|
|   IL|2020|  204|
|   IN|2005|  231|
|   KS|2020|  233|
+-----+----+-----+
only showing top 20 rows



Quels programmes d'assitance ont été activés dans chaque état ?

In [95]:
programs_columns = ["ih_program_declared", "ia_program_declared", "pa_program_declared", "hm_program_declared"]

disaster_df = data.withColumn("total_programs_activated", sum(when(col(c) == 1, 1).otherwise(0) for c in programs_columns))

#disaster_df.select("state", "fema_declaration_string", "total_programs_activated").show(truncate=False)

most_activated_program = disaster_df.groupBy("state").agg({"total_programs_activated": "max"}).withColumnRenamed("max(total_programs_activated)", "max_programs_activated")

most_activated_program.show()


+-----+----------------------+
|state|max_programs_activated|
+-----+----------------------+
|   SC|                     3|
|   AZ|                     3|
|   LA|                     4|
|   MN|                     4|
|   NJ|                     4|
|   DC|                     3|
|   OR|                     4|
|   VA|                     4|
|   RI|                     3|
|   KY|                     4|
|   WY|                     3|
|   NH|                     4|
|   MI|                     3|
|   NV|                     4|
|   WI|                     4|
|   ID|                     3|
|   CA|                     4|
|   CT|                     4|
|   NE|                     4|
|   MT|                     3|
+-----+----------------------+
only showing top 20 rows

